This is an example that shows how to use xgboost to handle audio classification. But it need to be modifid from multi-class to multi-label

In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import librosa
#from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score
from scipy.stats import skew
SAMPLE_RATE = 45100

#from sklearn.model_selection import KFold, RepeatedKFold
from tqdm import tqdm, tqdm_pandas

tqdm.pandas()
import scipy
data_path = '../input/'
ss = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))

['train_noisy.csv', 'test', 'train_curated', 'train_noisy', 'sample_submission.csv', 'train_curated.csv']


In [2]:
#loading data
audio_train_files = os.listdir('../input/train_curated/')
audio_test_files = os.listdir('../input/test/')

train = pd.read_csv('../input/train_curated.csv')
submission = pd.read_csv('../input/sample_submission.csv')

In [3]:
#function from EDA kernel: https://www.kaggle.com/codename007/a-very-extensive-freesound-exploratory-analysis
def clean_filename(fname, string):   
    file_name = fname.split('/')[1]
    if file_name[:2] == '__':        
        file_name = string + file_name
    return file_name

#returns mfcc features with mean and standard deviation along time
def get_mfcc(name, path):
    b, _ = librosa.core.load(path + name, sr = SAMPLE_RATE)
    assert _ == SAMPLE_RATE
    try:
        ft1 = librosa.feature.mfcc(b, sr = SAMPLE_RATE, n_mfcc=20)
        ft2 = librosa.feature.zero_crossing_rate(b)[0]
        ft3 = librosa.feature.spectral_rolloff(b)[0]
        ft4 = librosa.feature.spectral_centroid(b)[0]
        ft1_trunc = np.hstack((np.mean(ft1, axis=1), np.std(ft1, axis=1), skew(ft1, axis = 1), np.max(ft1, axis = 1), np.min(ft1, axis = 1)))
        ft2_trunc = np.hstack((np.mean(ft2), np.std(ft2), skew(ft2), np.max(ft2), np.min(ft2)))
        ft3_trunc = np.hstack((np.mean(ft3), np.std(ft3), skew(ft3), np.max(ft3), np.min(ft3)))
        ft4_trunc = np.hstack((np.mean(ft4), np.std(ft4), skew(ft4), np.max(ft4), np.min(ft4)))
        return pd.Series(np.hstack((ft1_trunc, ft2_trunc, ft3_trunc, ft4_trunc)))
    except:
        print('bad file')
        return pd.Series([0]*115)

In [9]:
#preparing data
train_data = pd.DataFrame()
train_data['fname'] = train['fname']
test_data = pd.DataFrame()
test_data['fname'] = audio_test_files

train_data = train_data['fname'].apply(get_mfcc, path='../input/train_curated/')
print('done loading train mfcc')
test_data = test_data['fname'].apply(get_mfcc, path='../input/test/')
print('done loading test mfcc')

train_data['fname'] = train['fname']
test_data['fname'] = audio_test_files

done loading train mfcc
done loading test mfcc


In [11]:
train_data['label'] = train['labels'].apply(lambda x:x.split(',')[0])
test_data['label'] = np.zeros((len(audio_test_files)))

In [12]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,fname,label
0,-505.089749,180.441064,15.068468,10.800259,-9.600303,4.576918,8.881082,-6.049447,-4.160875,4.630835,1.992888,0.560607,-0.090579,3.867188,3.349992,1.606435,1.705743,1.694710,5.780792,9.075267,67.172105,30.846796,23.057665,10.070208,16.333535,9.430675,8.262834,15.407742,10.851452,7.908980,8.867473,6.861082,7.182113,6.911124,6.462065,6.216631,7.572919,7.139398,6.598127,9.187224,...,30.056971,28.153337,43.951801,-736.838738,26.702456,-70.810188,-18.172710,-70.888891,-17.465429,-11.334218,-45.869613,-31.710518,-16.412176,-29.836455,-18.176302,-20.893256,-16.819023,-13.571973,-18.081521,-24.448299,-18.228786,-10.615964,-14.110874,0.022583,0.010618,0.751250,0.065918,0.000488,1167.300725,808.083240,2.531639,6093.896484,376.831055,745.175777,266.495106,3.284192,3017.984339,412.778244,0006ae4e.wav,Bark
1,-666.170512,89.343013,11.708950,44.341037,19.341729,23.879220,-3.023549,22.180218,11.925984,16.667709,-2.438549,5.341580,-0.705911,-6.309737,-6.273408,7.822523,2.538993,5.940230,4.128231,3.514395,111.003256,34.160382,27.391266,21.582060,16.461118,11.123822,10.640620,15.637882,9.534221,7.353127,6.592269,5.949958,9.484252,8.613264,7.694590,6.456034,5.835454,4.659840,5.759114,5.000570,...,19.719279,21.692740,15.149624,-910.170514,-90.945900,-89.207539,-8.936872,-36.500116,-1.623992,-46.019010,-12.112696,-17.606534,1.238040,-20.600297,-13.990914,-29.632725,-23.618353,-20.555851,-17.757781,-14.184980,-6.114079,-7.112447,-12.825725,0.064744,0.077409,3.144545,0.463379,0.003418,4408.953082,1302.057143,0.453779,7611.987305,1808.789062,1930.296240,805.582988,1.882097,5449.869552,938.740016,0019ef41.wav,Raindrop
2,-738.851927,24.688094,8.369442,10.640992,5.429327,4.093085,0.947911,0.124441,-0.381000,4.122988,1.677409,-0.168981,0.747731,1.216025,0.094377,-1.380938,-2.286875,-0.766164,-0.674148,-0.179731,113.765049,14.708152,22.546999,6.149522,5.795745,4.910861,4.428245,4.356917,5.998273,6.643124,3.508556,4.376543,2.779258,2.899820,2.409793,3.117367,3.568433,2.863102,2.675586,2.733805,...,10.958717,7.620899,12.061928,-785.183479,5.142378,-87.062206,-13.400298,-19.598588,-8.785371,-16.494646,-15.657153,-24.264134,-2.844079,-11.527364,-16.598262,-7.364427,-4.040663,-5.106069,-12.913259,-16.852777,-5.813879,-7.242315,-9.925327,0.294251,0.077837,-0.160418,0.475098,0.098145,6334.939803,689.305899,-2.475258,7159.790039,3445.312500,2637.285418,245.334940,-0.103139,3585.775617,1910.926095,001ec0ad.wav,Finger_snapping
3,-429.256141,89.789210,-3.267959,25.943623,-10.381607,5.995649,-5.471482,7.434913,0.204388,11.546948,1.724260,5.590036,2.697353,4.828801,4.021077,4.495891,1.704539,0.973502,-0.313691,4.303538,92.774764,35.785890,19.374310,11.614118,14.438876,10.936619,11.434155,7.377882,8.426847,6.186598,6.702710,5.934732,5.659035,5.009476,4.884049,4.892464,4.612518,4.396953,4.451472,4.430248,...,17.188587,13.986233,22.290690,-599.918235,9.567578,-95.874499,-22.919304,-61.543827,-35.481633,-45.069870,-29.306123,-32.703857,-9.403415,-38.398746,-16.327934,-19.206187,-20.469787,-19.553214,-12.346420,-15.052636,-16.458089,-14.184880,-10.601411,0.109377,0.054297,0.507787,0.255371,0.014160,5201.838784,1383.682232,-1.224612,7385.888672,656.762695,2419.062894,659.047645,-0.610302,4120.688332,783.909282,0026c7cb.wav,Run
4,-697.901080,7.609523,-2.702084,3.314518,1.075505,6.472180,1.506116,0.979527,0.642133,3.388375,0.368710,1.285987,0.282099,1.307120,2.080976,1.527842,-0.266796,1.709230,-0.576489,1.723662,154.435596,20.669898,14.534752,9.112657,6.169057,16.440526,5.880632,3.174800,4.244987,8.043572,5.713224,3.745933,4.225375,4.302439,5.589612,4.765563,3.869477,4.340709,4.579368,4.241854,...,17.325322,11.848079,19.295846,-751.576898,0.000000,-77.692454,-7.878032,-22.091396,-9.486487,-18.894384,-6.771509,-17.096250,0.0

In [13]:
def extract_features(files, path):
    features = {}
    cnt = 0
    for f in tqdm(files):
        features[f] = {}
        fs, data = scipy.io.wavfile.read(os.path.join(path, f))
        abs_data = np.abs(data)
        diff_data = np.diff(data)
        def calc_part_features(data, n=2, prefix=''):
            f_i = 1
            for i in range(0, len(data), len(data)//n):
                features[f]['{}mean_{}_{}'.format(prefix, f_i, n)] = np.mean(data[i:i + len(data)//n])
                features[f]['{}std_{}_{}'.format(prefix, f_i, n)] = np.std(data[i:i + len(data)//n])
                features[f]['{}min_{}_{}'.format(prefix, f_i, n)] = np.min(data[i:i + len(data)//n])
                features[f]['{}max_{}_{}'.format(prefix, f_i, n)] = np.max(data[i:i + len(data)//n])
        features[f]['len'] = len(data)
        if features[f]['len'] > 0:
            n = 1
            calc_part_features(data, n=n)
            calc_part_features(abs_data, n=n, prefix='abs_')
            calc_part_features(diff_data, n=n, prefix='diff_')

            n = 2
            calc_part_features(data, n=n)
            calc_part_features(abs_data, n=n, prefix='abs_')
            calc_part_features(diff_data, n=n, prefix='diff_')

            n = 3
            calc_part_features(data, n=n)
            calc_part_features(abs_data, n=n, prefix='abs_')
            calc_part_features(diff_data, n=n, prefix='diff_')
        cnt += 1
    features = pd.DataFrame(features).T.reset_index()
    features.rename(columns={'index': 'fname'}, inplace=True)
    return features

path = os.path.join(data_path, 'train_curated')
train_files = train.fname.values
train_features = extract_features(train_files, path)

path = os.path.join(data_path, 'test')
test_files = ss.fname.values
test_features = extract_features(test_files, path)





  0%|          | 0/4970 [00:00<?, ?it/s]



  0%|          | 4/4970 [00:00<03:09, 26.19it/s]



  0%|          | 11/4970 [00:00<02:46, 29.75it/s]



  0%|          | 16/4970 [00:00<02:31, 32.80it/s]



  0%|          | 19/4970 [00:00<02:57, 27.95it/s]



  1%|          | 30/4970 [00:00<02:17, 35.90it/s]



  1%|          | 38/4970 [00:00<02:04, 39.46it/s]



  1%|          | 43/4970 [00:00<01:59, 41.21it/s]



  1%|          | 48/4970 [00:01<02:10, 37.83it/s]



  1%|          | 53/4970 [00:01<02:08, 38.16it/s]



  1%|          | 61/4970 [00:01<01:49, 45.01it/s]



  1%|▏         | 67/4970 [00:01<01:50, 44.32it/s]



  1%|▏         | 72/4970 [00:01<01:53, 43.15it/s]



  2%|▏         | 77/4970 [00:01<02:16, 35.81it/s]



  2%|▏         | 82/4970 [00:02<02:39, 30.57it/s]



  2%|▏         | 86/4970 [00:02<02:31, 32.27it/s]



  2%|▏         | 94/4970 [00:02<02:04, 39.17it/s]



  2%|▏         | 101/4970 [00:02<01:51, 43.72it/s]



  2%|▏         | 107/4970 [00:02<01:44, 46.50it/s]


 14%|█▍        | 717/4970 [00:17<01:45, 40.27it/s]



 15%|█▍        | 722/4970 [00:17<01:52, 37.67it/s]



 15%|█▍        | 727/4970 [00:17<01:48, 39.12it/s]



 15%|█▍        | 734/4970 [00:17<01:36, 43.70it/s]



 15%|█▍        | 740/4970 [00:17<01:29, 47.19it/s]



 15%|█▌        | 746/4970 [00:17<01:44, 40.45it/s]



 15%|█▌        | 751/4970 [00:18<01:38, 42.74it/s]



 15%|█▌        | 758/4970 [00:18<01:29, 46.81it/s]



 15%|█▌        | 766/4970 [00:18<01:20, 52.41it/s]



 16%|█▌        | 772/4970 [00:18<01:28, 47.60it/s]



 16%|█▌        | 778/4970 [00:18<01:37, 43.00it/s]



 16%|█▌        | 783/4970 [00:18<01:42, 41.03it/s]



 16%|█▌        | 788/4970 [00:18<01:43, 40.56it/s]



 16%|█▌        | 795/4970 [00:18<01:33, 44.82it/s]



 16%|█▌        | 806/4970 [00:19<01:19, 52.20it/s]



 16%|█▋        | 814/4970 [00:19<01:14, 56.05it/s]



 17%|█▋        | 821/4970 [00:19<01:20, 51.24it/s]



 17%|█▋        | 827/4970 [00:19<01:19, 51.98it/s]



 17%|█▋        | 833/4970 [0

 29%|██▉       | 1444/4970 [00:34<01:29, 39.31it/s]



 29%|██▉       | 1449/4970 [00:34<01:32, 38.05it/s]



 29%|██▉       | 1454/4970 [00:34<01:36, 36.38it/s]



 29%|██▉       | 1463/4970 [00:34<01:21, 42.79it/s]



 30%|██▉       | 1471/4970 [00:34<01:20, 43.21it/s]



 30%|██▉       | 1476/4970 [00:34<01:35, 36.41it/s]



 30%|██▉       | 1481/4970 [00:34<01:28, 39.60it/s]



 30%|██▉       | 1486/4970 [00:35<01:31, 37.88it/s]



 30%|███       | 1494/4970 [00:35<01:24, 40.98it/s]



 30%|███       | 1499/4970 [00:35<01:20, 43.13it/s]



 30%|███       | 1504/4970 [00:35<01:41, 34.26it/s]



 30%|███       | 1508/4970 [00:35<01:48, 31.78it/s]



 30%|███       | 1515/4970 [00:35<01:40, 34.38it/s]



 31%|███       | 1522/4970 [00:36<01:27, 39.33it/s]



 31%|███       | 1527/4970 [00:36<01:22, 41.84it/s]



 31%|███       | 1532/4970 [00:36<01:44, 32.90it/s]



 31%|███       | 1536/4970 [00:36<01:43, 33.09it/s]



 31%|███       | 1540/4970 [00:36<01:50, 31.17it/s]



 31%|███  

 44%|████▎     | 2165/4970 [00:51<00:53, 52.90it/s]



 44%|████▎     | 2171/4970 [00:51<00:57, 48.55it/s]



 44%|████▍     | 2178/4970 [00:51<00:53, 51.87it/s]



 44%|████▍     | 2184/4970 [00:51<00:56, 49.08it/s]



 44%|████▍     | 2190/4970 [00:51<00:57, 48.16it/s]



 44%|████▍     | 2195/4970 [00:51<00:57, 48.25it/s]



 44%|████▍     | 2200/4970 [00:51<00:58, 47.24it/s]



 44%|████▍     | 2205/4970 [00:51<01:02, 44.38it/s]



 44%|████▍     | 2210/4970 [00:52<01:08, 40.12it/s]



 45%|████▍     | 2215/4970 [00:52<01:12, 37.96it/s]



 45%|████▍     | 2220/4970 [00:52<01:12, 37.96it/s]



 45%|████▍     | 2224/4970 [00:52<01:17, 35.55it/s]



 45%|████▍     | 2233/4970 [00:52<01:03, 43.31it/s]



 45%|████▌     | 2239/4970 [00:52<01:00, 45.06it/s]



 45%|████▌     | 2246/4970 [00:52<00:54, 50.20it/s]



 45%|████▌     | 2252/4970 [00:53<01:01, 44.19it/s]



 45%|████▌     | 2261/4970 [00:53<00:59, 45.69it/s]



 46%|████▌     | 2266/4970 [00:53<01:10, 38.40it/s]



 46%|████▌

 59%|█████▊    | 2919/4970 [01:08<00:46, 43.69it/s]



 59%|█████▉    | 2925/4970 [01:08<00:43, 47.14it/s]



 59%|█████▉    | 2931/4970 [01:08<00:42, 48.03it/s]



 59%|█████▉    | 2937/4970 [01:08<00:51, 39.85it/s]



 59%|█████▉    | 2942/4970 [01:09<01:04, 31.40it/s]



 59%|█████▉    | 2949/4970 [01:09<00:55, 36.66it/s]



 59%|█████▉    | 2954/4970 [01:09<00:56, 35.37it/s]



 60%|█████▉    | 2959/4970 [01:09<01:00, 33.47it/s]



 60%|█████▉    | 2963/4970 [01:09<01:03, 31.75it/s]



 60%|█████▉    | 2967/4970 [01:09<01:11, 27.86it/s]



 60%|█████▉    | 2972/4970 [01:09<01:02, 32.04it/s]



 60%|█████▉    | 2976/4970 [01:10<00:58, 33.90it/s]



 60%|█████▉    | 2980/4970 [01:10<00:58, 33.93it/s]



 60%|██████    | 2984/4970 [01:10<00:59, 33.65it/s]



 60%|██████    | 2992/4970 [01:10<00:52, 37.87it/s]



 60%|██████    | 2997/4970 [01:10<00:56, 34.85it/s]



 60%|██████    | 3001/4970 [01:10<01:08, 28.89it/s]



 60%|██████    | 3006/4970 [01:10<01:03, 31.07it/s]



 61%|█████

 73%|███████▎  | 3628/4970 [01:25<00:24, 55.86it/s]



 73%|███████▎  | 3634/4970 [01:25<00:32, 40.91it/s]



 73%|███████▎  | 3639/4970 [01:25<00:30, 43.14it/s]



 73%|███████▎  | 3649/4970 [01:25<00:27, 48.79it/s]



 74%|███████▎  | 3657/4970 [01:25<00:23, 55.09it/s]



 74%|███████▎  | 3664/4970 [01:26<00:26, 48.64it/s]



 74%|███████▍  | 3670/4970 [01:26<00:25, 51.52it/s]



 74%|███████▍  | 3678/4970 [01:26<00:25, 50.10it/s]



 74%|███████▍  | 3684/4970 [01:26<00:26, 48.20it/s]



 74%|███████▍  | 3691/4970 [01:26<00:25, 50.26it/s]



 74%|███████▍  | 3699/4970 [01:26<00:23, 53.91it/s]



 75%|███████▍  | 3705/4970 [01:26<00:29, 42.66it/s]



 75%|███████▍  | 3712/4970 [01:26<00:26, 47.68it/s]



 75%|███████▍  | 3718/4970 [01:27<00:28, 43.99it/s]



 75%|███████▍  | 3723/4970 [01:27<00:28, 43.18it/s]



 75%|███████▌  | 3728/4970 [01:27<00:28, 43.49it/s]



 75%|███████▌  | 3734/4970 [01:27<00:27, 45.76it/s]



 75%|███████▌  | 3739/4970 [01:27<00:26, 46.72it/s]



 75%|█████

 88%|████████▊ | 4388/4970 [01:42<00:14, 38.83it/s]



 88%|████████▊ | 4395/4970 [01:42<00:12, 44.71it/s]



 89%|████████▊ | 4400/4970 [01:42<00:18, 30.96it/s]



 89%|████████▊ | 4405/4970 [01:42<00:16, 33.64it/s]



 89%|████████▊ | 4410/4970 [01:42<00:15, 37.20it/s]



 89%|████████▉ | 4415/4970 [01:42<00:16, 33.83it/s]



 89%|████████▉ | 4423/4970 [01:43<00:13, 39.58it/s]



 89%|████████▉ | 4428/4970 [01:43<00:15, 35.97it/s]



 89%|████████▉ | 4433/4970 [01:43<00:16, 32.49it/s]



 89%|████████▉ | 4439/4970 [01:43<00:14, 37.38it/s]



 89%|████████▉ | 4446/4970 [01:43<00:12, 42.93it/s]



 90%|████████▉ | 4451/4970 [01:43<00:11, 43.32it/s]



 90%|████████▉ | 4461/4970 [01:43<00:09, 51.48it/s]



 90%|████████▉ | 4468/4970 [01:44<00:10, 49.99it/s]



 90%|█████████ | 4475/4970 [01:44<00:09, 54.20it/s]



 90%|█████████ | 4482/4970 [01:44<00:09, 53.93it/s]



 90%|█████████ | 4488/4970 [01:44<00:09, 49.13it/s]



 90%|█████████ | 4494/4970 [01:44<00:10, 45.83it/s]



 91%|█████

 14%|█▍        | 155/1120 [00:04<00:29, 32.92it/s]



 14%|█▍        | 159/1120 [00:04<00:34, 28.06it/s]



 15%|█▍        | 163/1120 [00:04<00:37, 25.42it/s]



 15%|█▌        | 170/1120 [00:04<00:30, 30.75it/s]



 16%|█▌        | 174/1120 [00:05<00:33, 28.40it/s]



 16%|█▌        | 179/1120 [00:05<00:28, 32.46it/s]



 17%|█▋        | 185/1120 [00:05<00:25, 37.26it/s]



 17%|█▋        | 190/1120 [00:05<00:31, 29.47it/s]



 18%|█▊        | 197/1120 [00:05<00:26, 34.97it/s]



 18%|█▊        | 202/1120 [00:05<00:29, 30.88it/s]



 19%|█▉        | 210/1120 [00:06<00:25, 35.89it/s]



 19%|█▉        | 215/1120 [00:06<00:27, 32.84it/s]



 20%|█▉        | 221/1120 [00:06<00:23, 37.86it/s]



 20%|██        | 228/1120 [00:06<00:21, 41.53it/s]



 21%|██        | 233/1120 [00:06<00:23, 38.16it/s]



 21%|██▏       | 238/1120 [00:06<00:28, 30.48it/s]



 22%|██▏       | 245/1120 [00:06<00:23, 36.54it/s]



 22%|██▏       | 250/1120 [00:07<00:23, 36.91it/s]



 23%|██▎       | 255/1120 [0

 68%|██████▊   | 760/1120 [00:22<00:13, 26.28it/s]



 68%|██████▊   | 766/1120 [00:22<00:11, 31.46it/s]



 69%|██████▉   | 770/1120 [00:22<00:11, 29.61it/s]



 69%|██████▉   | 777/1120 [00:22<00:09, 34.64it/s]



 70%|██████▉   | 782/1120 [00:22<00:10, 33.23it/s]



 70%|███████   | 786/1120 [00:22<00:09, 34.21it/s]



 71%|███████   | 790/1120 [00:23<00:10, 30.09it/s]



 71%|███████   | 796/1120 [00:23<00:09, 34.31it/s]



 72%|███████▏  | 802/1120 [00:23<00:08, 37.85it/s]



 72%|███████▏  | 807/1120 [00:23<00:09, 32.51it/s]



 72%|███████▏  | 811/1120 [00:23<00:09, 34.16it/s]



 73%|███████▎  | 815/1120 [00:23<00:09, 33.37it/s]



 73%|███████▎  | 822/1120 [00:23<00:07, 38.40it/s]



 74%|███████▍  | 827/1120 [00:24<00:10, 28.86it/s]



 74%|███████▍  | 831/1120 [00:24<00:11, 25.59it/s]



 75%|███████▍  | 836/1120 [00:24<00:09, 28.44it/s]



 75%|███████▌  | 840/1120 [00:24<00:09, 30.17it/s]



 75%|███████▌  | 845/1120 [00:24<00:08, 32.57it/s]



 76%|███████▌  | 851/1120 [0

In [14]:
train_data = train_data.merge(train_features, on='fname', how='left')
test_data = test_data.merge(test_features, on='fname', how='left')
train_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,114,fname,label,abs_max_1_1,abs_max_1_2,abs_max_1_3,abs_mean_1_1,abs_mean_1_2,abs_mean_1_3,abs_min_1_1,abs_min_1_2,abs_min_1_3,abs_std_1_1,abs_std_1_2,abs_std_1_3,diff_max_1_1,diff_max_1_2,diff_max_1_3,diff_mean_1_1,diff_mean_1_2,diff_mean_1_3,diff_min_1_1,diff_min_1_2,diff_min_1_3,diff_std_1_1,diff_std_1_2,diff_std_1_3,len,max_1_1,max_1_2,max_1_3,mean_1_1,mean_1_2,mean_1_3,min_1_1,min_1_2,min_1_3,std_1_1,std_1_2,std_1_3
0,-505.089749,180.441064,15.068468,10.800259,-9.600303,4.576918,8.881082,-6.049447,-4.160875,4.630835,1.992888,0.560607,-0.090579,3.867188,3.349992,1.606435,1.705743,1.694710,5.780792,9.075267,67.172105,30.846796,23.057665,10.070208,16.333535,9.430675,8.262834,15.407742,10.851452,7.908980,8.867473,6.861082,7.182113,6.911124,6.462065,6.216631,7.572919,7.139398,6.598127,9.187224,...,412.778244,0006ae4e.wav,Bark,3593.0,3593.0,2.0,260.261663,211.867614,2.0,0.0,0.0,2.0,314.171332,282.656719,0.0,633.0,1.0,633.0,0.000006,1.0,-0.001971,-688.0,1.0,-688.0,43.599811,0.0,56.619833,310456.0,3593.0,3593.0,2.0,-1.881023,-2.097250,2.0,-2891.0,-2891.0,2.0,407.965955,353.239732,0.0
1,-666.170512,89.343013,11.708950,44.341037,19.341729,23.879220,-3.023549,22.180218,11.925984,16.667709,-2.438549,5.341580,-0.705911,-6.309737,-6.273408,7.822523,2.538993,5.940230,4.128231,3.514395,111.003256,34.160382,27.391266,21.582060,16.461118,11.123822,10.640620,15.637882,9.534221,7.353127,6.592269,5.949958,9.484252,8.613264,7.694590,6.456034,5.835454,4.659840,5.759114,5.000570,...,938.740016,0019ef41.wav,Raindrop,4993.0,134.0,0.0,34.783239,16.208727,0.0,0.0,0.0,0.0,66.572596,20.219058,0.0,6905.0,0.0,32.0,-0.004072,0.0,-0.000397,-5490.0,0.0,-36.0,62.045847,0.0,1.491975,90616.0,4993.0,134.0,0.0,-0.614947,-0.836740,0.0,-3951.0,-129.0,0.0,75.109294,25.900444,0.0
2,-738.851927,24.688094,8.369442,10.640992,5.429327,4.093085,0.947911,0.124441,-0.381000,4.122988,1.677409,-0.168981,0.747731,1.216025,0.094377,-1.380938,-2.286875,-0.766164,-0.674148,-0.179731,113.765049,14.708152,22.546999,6.149522,5.795745,4.910861,4.428245,4.356917,5.998273,6.643124,3.508556,4.376543,2.779258,2.899820,2.409793,3.117367,3.568433,2.863102,2.675586,2.733805,...,1910.926095,001ec0ad.wav,Finger_snapping,21248.0,6.0,0.0,18.028577,0.764901,0.0,0.0,0.0,0.0,334.253144,0.718669,0.0,19286.0,0.0,6.0,0.000418,0.0,0.000045,-25905.0,0.0,-6.0,185.995498,0.0,0.433791,66976.0,18134.0,5.0,0.0,-0.479485,-0.497641,0.0,-21248.0,-6.0,0.0,334.738651,0.924074,0.0
3,-429.256141,89.789210,-3.267959,25.943623,-10.381607,5.995649,-5.471482,7.434913,0.204388,11.546948,1.724260,5.590036,2.697353,4.828801,4.021077,4.495891,1.704539,0.973502,-0.313691,4.303538,92.774764,35.785890,19.374310,11.614118,14.438876,10.936619,11.434155,7.377882,8.426847,6.186598,6.702710,5.934732,5.659035,5.009476,4.884049,4.892464,4.612518,4.396953,4.451472,4.430248,...,783.909282,0026c7cb.wav,Run,29301.0,29301.0,34.0,156.123777,168.922580,34.0,0.0,0.0,34.0,560.072413,624.067002,0.0,7476.0,-19.0,4856.0,0.000029,-19.0,0.000829,-9124.0,-19.0,-6492.0,107.378732,0.0,116.508453,1125886.0,28566.0,28566.0,34.0,0.000040,-0.334592,34.0,-29301.0,-29301.0,34.0,581.425611,646.524825,0.0
4,-697.901080,7.609523,-2.702084,3.314518,1.075505,6.472180,1.506116,0.979527,0.642133,3.388375,0.368710,1.285987,0.282099,1.307120,2.080976,1.527842,-0.266796,1.709230,-0.576489,1.723662,154.435596,20.669898,14.534752,9.112657,6.169057,16.440526,5.880632,3.174800,4.244987,8.043572,5.713224,3.745933,4.225375,4.302439,5.589612,4.765563,3.869477,4.340709,4.579368,4.241854,...,0.000000,0026f116.wav,Finger_snapping,28873.0,0.0,0.0,80.229081,0.000000,0.0,0.0,0.0,0.0,644.484197,0.000000,0.0,30680.0,0.0,0.0,0.000000,0.0,0.000000,-32629.0,0.0,0.0,476.771176,0.0,0.000000,60638.0,28873.0,0.0,0.0,0.023005,0.000000,0.0,-27956.0,0.0,0.0,649.458686,0.000000,0.0


In [15]:
#Functions from LightGBM baseline: https://www.kaggle.com/opanichev/lightgbm-baseline
# Construct features set
X = train_data.drop(['label', 'fname'], axis=1)
feature_names = list(X.columns)
X = X.values
labels = np.sort(np.unique(train_data.label.values))
num_class = len(labels)
c2i = {}
i2c = {}
for i, c in enumerate(labels):
    c2i[c] = i
    i2c[i] = c
y = np.array([c2i[x.split(',')[0]] for x in train_data.label.values])

78

In [33]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=10, shuffle = True)
clf = LGBMClassifier(max_depth=5, learning_rate=0.05, n_estimators=1000,
                    n_jobs=-1, random_state=0, reg_alpha=0.2, 
                    colsample_bylevel=0.5, colsample_bytree=0.5)
clf.fit(X_train, y_train)
print(accuracy_score(clf.predict(X_val), y_val))
#more functions from LightGBM baseline: https://www.kaggle.com/opanichev/lightgbm-baseline

0.45472837022132795


In [22]:
p = clf.predict_proba(test_data.drop(['label', 'fname'], axis =1).values)

In [26]:
p.shape

(1120, 78)

In [28]:
for i in range(len(labels)):
    submission[i2c[i]] = p[:, i]

In [29]:
submission.head()

,fname,Accelerating_and_revving_and_vroom,Accordion,Acoustic_guitar,Applause,Bark,Bass_drum,Bass_guitar,Bathtub_(filling_or_washing),Bicycle_bell,Burping_and_eructation,Bus,Buzz,Car_passing_by,Cheering,Chewing_and_mastication,Child_speech_and_kid_speaking,Chink_and_clink,Chirp_and_tweet,Church_bell,Clapping,Computer_keyboard,Crackle,Cricket,Crowd,Cupboard_open_or_close,Cutlery_and_silverware,Dishes_and_pots_and_pans,Drawer_open_or_close,Drip,Electric_guitar,Fart,Female_singing,Female_speech_and_woman_speaking,Fill_(with_liquid),Finger_snapping,Frying_(food),Gasp,Glockenspiel,Gong,...,Harmonica,Hi-hat,Hiss,Keys_jangling,Knock,Male_singing,Male_speech_and_man_speaking,Marimba_and_xylophone,Mechanical_fan,Meow,Microwave_oven,Motorcycle,Printer,Purr,Race_car_and_auto_racing,Raindrop,Run,Scissors,Screaming,Shatter,Sigh,Sink_(filling_or_washing),Skateboard,Slam,Sneeze,Squeak,Stream,Strum,Tap,Tick-tock,Toilet_flush,Traffic_noise_and_roadway_noise,Trickle_and_dribble,Walk_and_footsteps,Water_tap_and_faucet,Waves_and_surf,Whispering,Writing,Yell,Zipper_(clothing)
0,000ccb97.wav,0.026983,0.010287,0.010226,0.010723,0.010249,0.010736,0.010495,0.010190,0.010216,0.010239,0.023115,0.018187,0.036117,0.010165,0.010230,0.010218,0.010204,0.015536,0.022947,0.010395,0.015744,0.010199,0.012297,0.010794,0.011423,0.010233,0.010512,0.010328,0.010170,0.010312,0.010220,0.010818,0.010190,0.010188,0.010172,0.010375,0.010316,0.010167,0.010850,...,0.010204,0.010201,0.014631,0.010342,0.010583,0.010172,0.011242,0.010385,0.035212,0.011159,0.014607,0.016263,0.011424,0.030332,0.013748,0.010204,0.011556,0.010337,0.010177,0.010202,0.010201,0.010208,0.011273,0.010364,0.010226,0.011198,0.013807,0,0.011304,0.010457,0.010354,0.028564,0.010173,0.011770,0,0.021712,0.010418,0.011490,0.010175,0.010350
1,0012633b.wav,0.012510,0.011381,0.011424,0.011470,0.011500,0.011981,0.011377,0.011788,0.011433,0.011458,0.017397,0.029406,0.025846,0.011375,0.011448,0.011435,0.011419,0.015980,0.013130,0.011511,0.011514,0.011746,0.012836,0.012104,0.013734,0.011797,0.011421,0.012586,0.011381,0.013318,0.011455,0.011401,0.011404,0.011420,0.011383,0.011611,0.011451,0.011378,0.018449,...,0.011419,0.011416,0.011978,0.011455,0.012591,0.011384,0.012232,0.011545,0.011396,0.011925,0.015684,0.012376,0.012636,0.015937,0.013489,0.011419,0.013269,0.011683,0.011388,0.011417,0.012387,0.011424,0.012101,0.011871,0.011444,0.012035,0.014425,0,0.011749,0.011976,0.011446,0.023689,0.011385,0.017037,0,0.012090,0.012056,0.013421,0.011386,0.012359
2,001ed5f1.wav,0.011018,0.011465,0.011010,0.026148,0.011427,0.011799,0.011508,0.017791,0.011019,0.011809,0.012419,0.016346,0.011089,0.010964,0.011310,0.011283,0.012456,0.016562,0.012054,0.011175,0.011413,0.019380,0.011794,0.011115,0.010984,0.011724,0.016131,0.011156,0.011030,0.011396,0.011023,0.012626,0.015267,0.011949,0.010971,0.014878,0.010979,0.010966,0.017408,...,0.012299,0.027684,0.012654,0.011376,0.011123,0.011570,0.011277,0.017399,0.010984,0.014144,0.011052,0.011034,0.016410,0.011001,0.010998,0.019653,0.011078,0.019111,0.011999,0.011004,0.016580,0.011011,0.011161,0.012813,0.011030,0.011599,0.011066,0,0.011742,0.012469,0.011015,0.011170,0.011199,0.011655,0,0.011116,0.011197,0.012863,0.011158,0.011453
3,00294be0.wav,0.009614,0.009503,0.009496,0.010411,0.013096,0.011849,0.009391,0.009414,0.021746,0.009459,0.015398,0.010003,0.013421,0.009390,0.009583,0.009439,0.009429,0.010533,0.043291,0.009940,0.010523,0.010231,0.011296,0.014930,0.009408,0.009690,0.010100,0.009501,0.009395,0.009736,0.029091,0.011881,0.010234,0.009817,0.009397,0.009654,0.009403,0.009392,0.009478,...,0.009426,0.009424,0.009829,0.010210,0.009607,0.009609,0.010649,0.009960,0.012601,0.015342,0.012294,0.018364,0.015223,0.009713,0.023959,0.016469,0.016988,0.010394,0.009693,0.010400,0.009424,0.009471,0.016598,0.012780,0.011751,0.022758,0.013596,0,0.034094,0.021119,0.016110,0.032756,0.009398,0.010624,0,0.010081,0.009882,0.011531,0.009399,0.010675
4,003fde7a.wav,0.010799,0.010389,0.010446,0.010431,0.010469,0.010

In [34]:
submission.to_csv('submission.csv', index = False)

In [35]:
!ls

__notebook_source__.ipynb  submission.csv
